In [ ]:
!pip install googlesearch-python

from googlesearch import search
import requests
from bs4 import BeautifulSoup
import re
import json
import time
from IPython.display import FileLink

def limpiar_nombre(raw_title):
    if not raw_title:
        return "Nombre no encontrado"
    nombre = raw_title.strip()

    # Eliminar textos comunes no deseados y truncar tras ciertos separadores
    nombre = re.split(r'[-–—|:•]', nombre)[0]
    nombre = re.sub(r'\b(contacto|contactos|página oficial|home|portal|web|site|oficial)\b', '', nombre, flags=re.I)
    nombre = re.sub(r'\d{1,4}[\s,.-]*[a-zA-Z]*', '', nombre)
    nombre = nombre.strip()

    # Capitalizar cada palabra
    nombre = ' '.join(w.capitalize() for w in nombre.split())

    if not nombre:
        return "Nombre no encontrado"
    return nombre

busquedas = [
  "colegios en ecuador contacto email",
    "escuelas en ecuador correo de contacto",
    "unidad educativa ecuador email",
    "colegio particular ecuador correo",
    "colegio oficial ecuador contacto email",
    "institución educativa ecuador email",
    "colegios bilingües ecuador contacto",
    "colegio católico ecuador correo electrónico",
    "colegio privado ecuador contacto",
    "escuela pública ecuador correo",
    "colegios urbanos ecuador email",
    "colegios rurales ecuador email contacto",
    "unidad educativa particular ecuador email",
    "colegios con email de contacto ecuador",
    "escuelas con email de contacto ecuador",
    "colegios bilingües privados ecuador correo",
    "colegio secundario ecuador contacto email",
    "colegio primaria ecuador email",
    "colegio técnico ecuador correo electrónico",
    "colegio religioso ecuador contacto email",
    "colegios oficiales ecuador email",
    "colegios católicos ecuador correo electrónico",
    "colegios privados en quito email",
    "colegios privados en guayaquil contacto email",
    "unidad educativa rural ecuador correo electrónico",
    "colegio con correo oficial ecuador",
    "institución educativa con email contacto ecuador",
]

resultados = []
correos_vistos = set()
MAX_RESULTADOS = 1000

for query in busquedas:
    print(f"Buscando: {query}")
    try:
        for url in search(query, num_results=20, lang="es"):
            if len(resultados) >= MAX_RESULTADOS:
                break
            try:
                res = requests.get(url, timeout=10)
                res.raise_for_status()
                soup = BeautifulSoup(res.text, "html.parser")
                texto = soup.get_text(separator=' ')
                # Extraer solo correos terminados en .edu.ec
                correos_raw = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(edu\.ec)", texto, flags=re.I)
                correos = set(m.group(0) if hasattr(m,'group') else m for m in re.finditer(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.(edu\.ec)", texto, flags=re.I))
                correos = {c.group(0).lower() for c in re.finditer(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.edu\.ec", texto, flags=re.I)}
                correos = {c for c in correos if c not in correos_vistos}

                if not correos:
                    continue

                nombre_raw = soup.title.string if soup.title else None
                nombre = limpiar_nombre(nombre_raw)

                # Evitar que agregue si ya existe nombre igual (puedes optar si quieres unir correos)
                if any(r['nombre'] == nombre for r in resultados):
                    # Si ya está, agregar correos únicos a ese registro
                    for r in resultados:
                        if r['nombre'] == nombre:
                            nuevos = [c for c in correos if c not in r['correos'].split(', ')]
                            if nuevos:
                                r['correos'] += ', ' + ', '.join(nuevos)
                                correos_vistos.update(nuevos)
                    continue

                correos_vistos.update(correos)

                resultado = {
                    "nombre": nombre,
                    "correos": ", ".join(sorted(correos))
                }
                resultados.append(resultado)
                print(f"  + {nombre}: {len(correos)} correos")
                time.sleep(1)
            except Exception as e:
                print(f"Error en {url}: {e}")
    except Exception as e:
        print(f"Error en búsqueda '{query}': {e}")
    if len(resultados) >= MAX_RESULTADOS:
        break

print(f"\nTotal resultados: {len(resultados)}")

# Guardar en JSON
nombre_archivo = "colegios_ecuador.json"
with open(nombre_archivo, "w", encoding="utf-8") as f:
    f.write("[\n")
    for i, item in enumerate(resultados):
        coma = "," if i < len(resultados)-1 else ""
        json_str = json.dumps(item, ensure_ascii=False)
        f.write(f"  {json_str}{coma}\n")
    f.write("]\n")

print(f"\nArchivo '{nombre_archivo}' guardado con éxito.")

# Mostrar enlace para descargar en Colab
display(FileLink(nombre_archivo))

# Mostrar una muestra de resultados en consola (los 10 primeros)
print("\nEjemplo de resultados (10 primeros):")
for item in resultados[:10]:
    print(json.dumps(item, ensure_ascii=False, indent=2))


Buscando: colegios en ecuador contacto email
  + Nombre no encontrado: 7 correos
  + Y Ubicación: 27 correos
Error en https://www.casc.edu.ec/directorio/: HTTPSConnectionPool(host='www.casc.edu.ec', port=443): Max retries exceeded with url: /directorio/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)')))
  + Contáctenos: 1 correos
Error en https://www.colegiomenor.edu.ec/es/contact-us: 429 Client Error: Too Many Requests for url: https://www.colegiomenor.edu.ec/es/contact-us
Error en https://www.educacionsuperior.gob.ec/wp-content/uploads/downloads/2014/09/Directorio-IES-2014.pdf: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error en https://becquerel.edu.ec/contacto/: 403 Client Error: Forbidden for url: https://becquerel.edu.ec/contacto/
Error en https://www.isaacnewton.edu.ec/contacto/: 403 Client Error: Forbidden for url: 

/content/colegios_ecuador.json


Ejemplo de resultados (10 primeros):
{
  "nombre": "Nombre no encontrado",
  "correos": "cbonilla@colegiodeliga.edu.ec, cflores@colegiodeliga.edu.ec, fcabascango@colegiodeliga.edu.ec, jnaranjo@colegiodeliga.edu.ec, mesadeayuda@colegiodeliga.edu.ec, secretariageneral@colegiodeliga.edu.ec, vvaca@colegiodeliga.edu.ec, info@colegioterranova.edu.ec, info@seneca.edu.ec, administracion@seneca.edu.ec, inspeccion@colegiodeamerica.edu.ec, secretaria@colegiodeamerica.edu.ec, colecturia@colegiodeamerica.edu.ec, trabajosocial@colegiodeamerica.edu.ec, info@sanagustin.edu.ec, admisiones@sanagustin.edu.ec, n@colrosarioquito.edu.ec, rosarista@colrosario.edu.ec, informacion@colrosarioquito.edu.ec, info@thomasmore.edu.ec, magui.montalvo@soyamericalatina.edu.ec, maribel.quinteros@soyamericalatina.edu.ec, admisiones@ceeal.edu.ec, colecturia@soyamericalatina.edu.ec, uep@copol.edu.ec, colecturia@copol.edu.ec, admisiones@copol.edu.ec, asistentederectorado@copol.edu.ec, admisiones@williamthomsoninternacional.